In [1]:
import pandas as pd
import numpy as np

In [2]:
logsTrocafone = pd.read_csv('Data/events_up_to_01062018.csv', low_memory = False)

#trainingSet = pd.read_csv('trainingSet.csv')
#trainingSet = trainingSet.set_index('person')

#testingSet = pd.read_csv('testingSet.csv')
#testingSet = testingSet.set_index('person')

#trainingSet.head()

In [3]:
df = logsTrocafone

In [4]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [5]:
df.sort_values(by=['timestamp']).sort_values(by=['person'], inplace = True)

In [6]:
df['search_term'] = df['search_term'].str.lower()
df['model'] = df['model'].str.lower()

In [7]:
df['search_term'] = df['search_term'].str.replace('!','').str.replace('?','').str.replace('¡','').str.replace('¿','').str.replace('+','').str.replace('-','').str.replace('*','').str.replace('%','').str.replace(',','').str.replace(';','').str.replace('.','').str.replace(':','').str.replace('_','').str.replace('\'','').str.replace('\"','').str.replace('\\','').str.replace('/','').str.replace('(','').str.replace(')','').str.replace('{','').str.replace('}','').str.replace('[','').str.replace(']','').str.replace('@','').str.replace('º','').str.replace('&','')

In [8]:
dfo = df

In [24]:
df = dfo
df['timediff'] = df['timestamp'].diff()
df['timediff'] = df['timediff'].dt.total_seconds()

df['newperson']= df['person'].apply(lambda x: int(x, 16))
df['newperson'] = df['newperson'].diff()
df['prueba'] = df['newperson']
df['newperson'] = ((df['newperson'] != 0.0) | (df.index == 0))

maxtime = 16368
df['newentry'] = ((df['timediff'] > maxtime) | (df['newperson']))
#df['entry'] = df['newentry'].cumsum()
df['newsearch'] = ((df['event'] == 'searched products') | (df['newentry']))
df['validsearch'] = ((df['event'] == 'searched products') & (df['search_term']))
df['search'] = df['newsearch'].cumsum()

In [30]:
print(df.loc[:40, ['person', 'search', 'prueba', 'newperson', 'newentry', 'newsearch', 'validsearch']])

      person  search        prueba  newperson  newentry  newsearch  \
5   4c8a8b93       6 -2.151947e+09       True      True       True   
18  29ebb414      16 -9.904237e+08       True      True       True   
29  15ea8012      26 -2.632148e+09       True      True       True   
30  15ea8012      26  0.000000e+00      False     False      False   
35  d21b8e6b      31  2.812887e+09       True      True       True   
39  d21b8e6b      35  3.157332e+09       True      True       True   

    validsearch  
5          True  
18         True  
29         True  
30        False  
35         True  
39         True  


In [26]:
df = df.groupby('search').filter(lambda x: x.iloc[0]['validsearch'])

In [27]:
df_filtered = df

In [34]:
print(df.loc[:50, ['person', 'search', 'timediff', 'event', 'newentry', 'newsearch', 'validsearch']])

      person  search  timediff              event  newentry  newsearch  \
5   4c8a8b93       6    1998.0  searched products      True       True   
18  29ebb414      16     -18.0  searched products      True       True   
29  15ea8012      26    -318.0  searched products      True       True   
30  15ea8012      26      -4.0     viewed product     False      False   
35  d21b8e6b      31     -18.0  searched products      True       True   
39  d21b8e6b      35      -1.0  searched products      True       True   

    validsearch  
5          True  
18         True  
29         True  
30        False  
35         True  
39         True  


In [35]:
df.head()

,timestamp,event,person,url,sku,model,condition,storage,color,skus,...,screen_resolution,operating_system_version,browser_version,timediff,newperson,newentry,newsearch,validsearch,search,prueba
5,2018-05-18 00:44:27,searched products,4c8a8b93,NaN,NaN,NaN,NaN,NaN,NaN,"10240,9987,10322,10085,9944,9931,13404,10154,1...",...,NaN,NaN,NaN,1998.0,True,True,True,True,6,-2.151947e+09
18,2018-05-18 00:37:16,searched products,29ebb414,NaN,NaN,NaN,NaN,NaN,NaN,"3371,6357,6371,3191,2718,6001,2777,6248,2725,8...",...,NaN,NaN,NaN,-18.0,True,True,True,True,16,-9.904237e+08
29,2018-05-18 00:29:48,searched products,15ea8012,NaN,NaN,NaN,NaN,NaN,NaN,"12786,12800,12772",...,NaN,NaN,NaN,-318.0,True,True,True,True,26,-2.632148e+09
30,2018-05-18 00:29:44,viewed product,15ea8012,NaN,12800.0,samsung galaxy s8 plus,Bom,64GB,Prata,NaN,...,NaN,NaN,NaN,-4.0,False,False,False,False,26,0.000000e+00
35,2018-05-18 00:28:57,searched products,d21b8e6b,NaN,NaN,NaN,NaN,NaN,NaN,"5016,5088,8397,8429,10728,5075,7631,9216,4969,...",...,NaN,NaN,NaN,-18.0,True,True,True,True,31,2.812887e+09


Creo un DataFrame con una columna llamada 'busquedas' que contiene la marca buscada a partir del evento 'searched products' hasta otra busqueda o el fin de la sesión.  
Le agrego la columna 'nmodel' que contiene la marca que figura en 'model' de modo de poder comparar con 'busquedas'  
Agrego las columnas siguientes que pueden tomar como valor a 0 o 1:  
**bcm**: Busqueda con marca: es 1 si habia una marca en los términos de búsqueda  
**vam**: Visitas a marca: es 1 si tras la búsqueda de una marca se visita un producto de la misma  
**vao**: Visitas a otra marca: es 1 si tras la búsqueda de una marca se visita un producto de otra marca  
**coam**: Checkouts a marca: es 1 si tras la búsqueda de una marca se hace un checkout con un producto de la misma  
**coao**: Checkouts a otra marca: es 1 si tras la búsqueda de una marca se hace un checkout con un producto de otra marca  
**cam**: Conversión a marca: es 1 si tras la búsqueda de una marca se hace una conversion con un producto de la misma  
**cao**: Conversión a otra marca: es 1 si tras la búsqueda de una marca se hace una conversion con un producto de otra marca  
**lam**: Lead a marca: es 1 si tras la búsqueda de una marca se hace un lead con un producto de la misma  
**lao**: Lead a otra marca: es 1 si tras la búsqueda de una marca se hace un lead con un producto de otra marca

In [105]:
df = df_filtered
df2 = df

def contenido(s,x):
    return x.find(s) >= 0

def obtener_marca(x):
    if (type(x) == float):
        return np.nan
    elif (contenido('asus',x) or contenido('live',x) or contenido('zenfone',x)):
        return 'asus'
    elif (contenido('blackberry',x)):
        return 'blackberry'
    elif (contenido('lg',x)):
        return 'lg'
    elif (contenido('lenovo',x) or contenido('vibe',x)):
        return 'lenovo'
    elif (contenido('moto',x)):
        return 'motorola'
    elif (contenido('quantum',x)):
        return 'quantum'
    elif (contenido('samsung',x) or contenido('galaxy',x)):
        return 'samsung'
    elif (contenido('sony',x) or contenido('xperia',x)):
        return 'sony'
    elif (contenido('apple',x) or contenido('iphone',x) or contenido('ipad',x)):
        return 'apple'
    else:
        return np.nan

def obtener_marca_de_busqueda(x):
    return obtener_marca(x.iloc[0])

dfg = df.groupby('search', as_index=False)
df2['busquedas'] = dfg['search_term'].transform(obtener_marca_de_busqueda)
df2['nmodel'] = df2.apply(lambda x: obtener_marca(x['model']), axis=1)

df2['bcm'] = ((df2['event'] == 'searched products') & (df2['busquedas'])).astype(int)
df2['vam'] = ((df2['event'] == 'viewed product') & (df2['busquedas']) & (df2['busquedas'] == df2['nmodel'])).astype(int)
df2['vao'] = ((df2['event'] == 'viewed product') & (df2['busquedas']) & (df2['busquedas'] != df2['nmodel'])).astype(int)
df2['coam'] = ((df2['event'] == 'checkout') & (df2['busquedas']) & (df2['busquedas'] == df2['nmodel'])).astype(int)
df2['coao'] = ((df2['event'] == 'checkout') & (df2['busquedas']) & (df2['busquedas'] != df2['nmodel'])).astype(int)
df2['cam'] = ((df2['event'] == 'conversion') & (df2['busquedas']) & (df2['busquedas'] == df2['nmodel'])).astype(int)
df2['cao'] = ((df2['event'] == 'conversion') & (df2['busquedas']) & (df2['busquedas'] != df2['nmodel'])).astype(int)
df2['lam'] = ((df2['event'] == 'lead') & (df2['busquedas']) & (df2['busquedas'] == df2['nmodel'])).astype(int)
df2['lao'] = ((df2['event'] == 'lead') & (df2['busquedas']) & (df2['busquedas'] != df2['nmodel'])).astype(int)

print(df2.loc[:1000, ['search', 'event', 'search_term', 'model', 'nmodel', 'busquedas', 'bcm', 'vam', 'vao', 'coam', 'coao', 'cam', 'cao', 'lam', 'lao']])

/home/sebastian/.local/lib/python2.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/.local/lib/python2.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/sebastian/.local/lib/python2.7/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/sta

     search              event         search_term  \
5         6  searched products           iphone se   
18       16  searched products              6splus   
29       26  searched products      galaxy s8 plus   
30       26     viewed product                 NaN   
35       31  searched products             moto g3   
39       35  searched products             moto g5   
136     110  searched products                  5c   
153     125  searched products       iphone 7 plus   
158     130  searched products            iphone 6   
166     135  searched products          samsung s6   
175     142  searched products          samsung s6   
176     143  searched products          samsung s7   
177     144  searched products          samsung s7   
183     149  searched products              j5 pro   
212     174  searched products                  j5   
222     184  searched products            motorola   
235     194  searched products              j5 pto   
251     208  searched produc

In [106]:
print(df2.loc[:400, ['person', 'search', 'event', 'nmodel', 'busquedas', 'bcm', 'vam', 'vao', 'coam', 'coao', 'cam', 'cao', 'lam', 'lao']])

       person  search              event   nmodel busquedas  bcm  vam  vao  \
5    4c8a8b93       6  searched products      NaN     apple    1    0    0   
18   29ebb414      16  searched products      NaN       NaN    0    0    0   
29   15ea8012      26  searched products      NaN   samsung    1    0    0   
30   15ea8012      26     viewed product  samsung   samsung    0    1    0   
35   d21b8e6b      31  searched products      NaN  motorola    1    0    0   
39   d21b8e6b      35  searched products      NaN  motorola    1    0    0   
136  f87be219     110  searched products      NaN       NaN    0    0    0   
153  e2bfe05f     125  searched products      NaN     apple    1    0    0   
158  cb68850c     130  searched products      NaN     apple    1    0    0   
166  9bb3af27     135  searched products      NaN   samsung    1    0    0   
175  9bb3af27     142  searched products      NaN   samsung    1    0    0   
176  9bb3af27     143  searched products      NaN   samsung    1

In [107]:
print(df2.loc[df2['person'] == '15ea8012', ['person', 'search', 'event', 'nmodel', 'busquedas', 'bcm', 'vam', 'vao', 'coam', 'coao', 'cam', 'cao', 'lam', 'lao']])

          person  search              event   nmodel busquedas  bcm  vam  vao  \
29      15ea8012      26  searched products      NaN   samsung    1    0    0   
30      15ea8012      26     viewed product  samsung   samsung    0    1    0   
365     15ea8012     306  searched products      NaN   samsung    1    0    0   
200136  15ea8012  158220  searched products      NaN   samsung    1    0    0   
200166  15ea8012  158247  searched products      NaN   samsung    1    0    0   
261749  15ea8012  204532  searched products      NaN   samsung    1    0    0   
262269  15ea8012  204952  searched products      NaN   samsung    1    0    0   

        coam  coao  cam  cao  lam  lao  
29         0     0    0    0    0    0  
30         0     0    0    0    0    0  
365        0     0    0    0    0    0  
200136     0     0    0    0    0    0  
200166     0     0    0    0    0    0  
261749     0     0    0    0    0    0  
262269     0     0    0    0    0    0  


### A partir del df2 creo los features

In [75]:
dftmp = df2[(df2[['bcm']] == 1).all(axis=1)]
dftmp = dftmp.loc[:, ['person', 'busquedas', 'bcm']]
print(dftmp.loc[dftmp['person'] == 'ff77d8a0', :])

          person busquedas  bcm
23454   ff77d8a0     apple    1
47270   ff77d8a0     apple    1
47528   ff77d8a0     apple    1
74074   ff77d8a0     apple    1
74173   ff77d8a0     apple    1
74265   ff77d8a0     apple    1
76283   ff77d8a0     apple    1
115127  ff77d8a0     apple    1
115492  ff77d8a0     apple    1
118531  ff77d8a0     apple    1
119379  ff77d8a0     apple    1
119396  ff77d8a0     apple    1
140535  ff77d8a0     apple    1
141077  ff77d8a0     apple    1
141127  ff77d8a0     apple    1
141128  ff77d8a0     apple    1
142057  ff77d8a0     apple    1
142277  ff77d8a0     apple    1
142294  ff77d8a0     apple    1
142295  ff77d8a0     apple    1
142403  ff77d8a0     apple    1
142535  ff77d8a0     apple    1
174016  ff77d8a0  motorola    1
182926  ff77d8a0     apple    1
183065  ff77d8a0     apple    1
183220  ff77d8a0     apple    1
220563  ff77d8a0     apple    1
220713  ff77d8a0     apple    1
220715  ff77d8a0     apple    1
220750  ff77d8a0     apple    1
220773  

In [76]:
dftmp = dftmp.pivot_table(index='person',columns='busquedas',aggfunc=sum)
#print(dftmp.loc[dftmp['person'] == '00546f4c', :])
dftmp.fillna(0,inplace=True)
dftmp.columns = dftmp.columns.droplevel()
dftmp.columns.name = None
dftmp.reset_index(inplace=True)

,person,apple,asus,blackberry,lenovo,lg,motorola,quantum,samsung,sony
0,000c79fe,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
1,000e619d,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,001001be,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,001802e4,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0019e639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
5,0027574e,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,002aea56,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,002e74b2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,002ed810,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0031c75f,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
dftmp['person'].is_unique

In [ ]:
dftmp = dftmp.set_index('person')

In [93]:
dftmp

,apple,asus,blackberry,lenovo,lg,motorola,quantum,samsung,sony
person,,,,,,,,,
000c79fe,6,0,0,0,0,0,0,2,0
000e619d,0,0,0,0,0,0,0,1,0
001001be,12,0,0,0,0,0,0,0,0
001802e4,2,0,0,0,0,0,0,0,0
0019e639,0,0,0,0,0,0,0,3,0
0027574e,0,1,0,0,0,0,0,0,0
002aea56,1,0,0,0,0,0,0,0,0
002e74b2,2,0,0,0,0,0,0,0,0
002ed810,2,0,0,0,0,0,0,0,0


In [94]:
trainingSet = pd.read_csv('trainingSet.csv')
trainingSet = trainingSet.set_index('person')

testingSet = pd.read_csv('testingSet.csv')
testingSet = testingSet.set_index('person')

trainingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,CantidadSearchEngineHits,EntraronMasDeUnaVez,DiasDesdeUltimoCheckout,EntraronConComputadora,EntraronConSmartphone,EntraronConTablet,CantidadAdHits,DiasDesdeUltimoAdHit,DiasDesdeUltimaBusqueda,CantidadDeColoresVistos
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,6,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0.0,1.0,15.0,1.0,0.0,0.0,0.0,1000.0,1000.0,3
000c79fe,17,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,3.0,0.0,1.0,0.0,1.0,3.0,3.0,1
001802e4,19,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,5.0,1.0,1.0,2
0019e639,471,2.0,0.0,1.0,0.0,1.0,0.0,15.0,0.0,1.0,...,13.0,1.0,10.0,1.0,0.0,0.0,29.0,10.0,10.0,14
001b0bf9,7,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,1000.0,1000.0,2


In [95]:
nuevoTrainingSet = trainingSet.merge(dftmp, how='left', left_index=True, right_index=True)

nuevoTrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, 0008ed71 to fff78145
Data columns (total 35 columns):
CantidadEventos              19414 non-null int64
CantidadCompras              19414 non-null float64
CantidadLeads                19414 non-null float64
VentasHaceUnMes              19414 non-null float64
LeadsHaceUnMes               19414 non-null float64
VentasHaceDosMeses           19414 non-null float64
LeadsHaceDosMeses            19414 non-null float64
CantidadCheckouts            19414 non-null float64
VentasMes1                   19414 non-null float64
VentasMes2                   19414 non-null float64
VentasMes3                   19414 non-null float64
VentasMes4                   19414 non-null float64
PromedioVentasPorMes         19414 non-null float64
DiasDesdeUltimoEvento        19414 non-null float64
DiasDesdePrimerEvento        19414 non-null float64
DiasDesdeUltimaConversion    19414 non-null float64
CantidadSearchEngineHits     19414 non-null float64
Entr

In [97]:
nuevoTrainingSet['apple'].fillna(0, inplace=True)
nuevoTrainingSet['asus'].fillna(0, inplace=True)
nuevoTrainingSet['blackberry'].fillna(0, inplace=True)
nuevoTrainingSet['lenovo'].fillna(0, inplace=True)
nuevoTrainingSet['lg'].fillna(0, inplace=True)
nuevoTrainingSet['motorola'].fillna(0, inplace=True)
nuevoTrainingSet['quantum'].fillna(0, inplace=True)
nuevoTrainingSet['samsung'].fillna(0, inplace=True)
nuevoTrainingSet['sony'].fillna(0, inplace=True)

nuevoTrainingSet['apple'] = nuevoTrainingSet['apple'].astype(int)
nuevoTrainingSet['asus'] = nuevoTrainingSet['asus'].astype(int)
nuevoTrainingSet['blackberry'] = nuevoTrainingSet['blackberry'].astype(int)
nuevoTrainingSet['lenovo'] = nuevoTrainingSet['lenovo'].astype(int)
nuevoTrainingSet['lg'] = nuevoTrainingSet['lg'].astype(int)
nuevoTrainingSet['motorola'] = nuevoTrainingSet['motorola'].astype(int)
nuevoTrainingSet['quantum'] = nuevoTrainingSet['quantum'].astype(int)
nuevoTrainingSet['samsung'] = nuevoTrainingSet['samsung'].astype(int)
nuevoTrainingSet['sony'] = nuevoTrainingSet['sony'].astype(int)
nuevoTrainingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,CantidadDeColoresVistos,apple,asus,blackberry,lenovo,lg,motorola,quantum,samsung,sony
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,6,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,3,0,0,0,0,0,0,0,0,0
000c79fe,17,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1,6,0,0,0,0,0,0,2,0
001802e4,19,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,2,2,0,0,0,0,0,0,0,0
0019e639,471,2.0,0.0,1.0,0.0,1.0,0.0,15.0,0.0,1.0,...,14,0,0,0,0,0,0,0,3,0
001b0bf9,7,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,...,2,0,0,0,0,0,0,0,0,0


In [98]:
nuevoTestingSet = testingSet.merge(dftmp, how='left', left_index=True, right_index=True)

nuevoTestingSet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19415 entries, 00091926 to fffd1246
Data columns (total 35 columns):
CantidadEventos              19415 non-null int64
CantidadCompras              19415 non-null float64
CantidadLeads                19415 non-null float64
VentasHaceUnMes              19415 non-null float64
LeadsHaceUnMes               19415 non-null float64
VentasHaceDosMeses           19415 non-null float64
LeadsHaceDosMeses            19415 non-null float64
CantidadCheckouts            19415 non-null float64
VentasMes1                   19415 non-null float64
VentasMes2                   19415 non-null float64
VentasMes3                   19415 non-null float64
VentasMes4                   19415 non-null float64
PromedioVentasPorMes         19415 non-null float64
DiasDesdeUltimoEvento        19415 non-null float64
DiasDesdePrimerEvento        19415 non-null float64
DiasDesdeUltimaConversion    19415 non-null float64
CantidadSearchEngineHits     19415 non-null float64
Entr

In [99]:
nuevoTestingSet['apple'].fillna(0, inplace=True)
nuevoTestingSet['asus'].fillna(0, inplace=True)
nuevoTestingSet['blackberry'].fillna(0, inplace=True)
nuevoTestingSet['lenovo'].fillna(0, inplace=True)
nuevoTestingSet['lg'].fillna(0, inplace=True)
nuevoTestingSet['motorola'].fillna(0, inplace=True)
nuevoTestingSet['quantum'].fillna(0, inplace=True)
nuevoTestingSet['samsung'].fillna(0, inplace=True)
nuevoTestingSet['sony'].fillna(0, inplace=True)

nuevoTestingSet['apple'] = nuevoTestingSet['apple'].astype(int)
nuevoTestingSet['asus'] = nuevoTestingSet['asus'].astype(int)
nuevoTestingSet['blackberry'] = nuevoTestingSet['blackberry'].astype(int)
nuevoTestingSet['lenovo'] = nuevoTestingSet['lenovo'].astype(int)
nuevoTestingSet['lg'] = nuevoTestingSet['lg'].astype(int)
nuevoTestingSet['motorola'] = nuevoTestingSet['motorola'].astype(int)
nuevoTestingSet['quantum'] = nuevoTestingSet['quantum'].astype(int)
nuevoTestingSet['samsung'] = nuevoTestingSet['samsung'].astype(int)
nuevoTestingSet['sony'] = nuevoTestingSet['sony'].astype(int)
nuevoTestingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,CantidadDeColoresVistos,apple,asus,blackberry,lenovo,lg,motorola,quantum,samsung,sony
person,,,,,,,,,,,,,,,,,,,,,
00091926,448,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,20,0,0,0,0,0,0,0,0,0
00091a7a,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3,0,0,0,0,0,0,0,0,0
000ba417,206,1.0,0.0,1.0,0.0,1.0,0.0,6.0,0.0,0.0,...,20,0,0,0,0,0,0,0,0,0
000e4d9e,411,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,12,0,0,0,0,0,0,0,0,0
000e619d,68,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,9,0,0,0,0,0,0,0,1,0


In [100]:
#nuevoTrainingSet.to_csv('trainingSet2.csv')
#Este comando reemplaza el training set existente con la feature agregada

In [101]:
#nuevoTestingSet.to_csv('testingSet2.csv')
#Este comando reemplaza el testing set existente con la feature agregada

In [109]:
dftmp = df2.loc[:, ['person', 'bcm', 'vam', 'vao', 'coam', 'coao', 'cam', 'cao', 'lam', 'lao']]
dftmp.head()

,person,bcm,vam,vao,coam,coao,cam,cao,lam,lao
5,4c8a8b93,1,0,0,0,0,0,0,0,0
18,29ebb414,0,0,0,0,0,0,0,0,0
29,15ea8012,1,0,0,0,0,0,0,0,0
30,15ea8012,0,1,0,0,0,0,0,0,0
35,d21b8e6b,1,0,0,0,0,0,0,0,0


In [111]:
dftmp = dftmp.groupby('person', as_index=True).sum()
dftmp.head()

,bcm,vam,vao,coam,coao,cam,cao,lam,lao
person,,,,,,,,,
000c79fe,8,0,0,0,0,0,0,0,0
000e619d,1,0,0,0,0,0,0,0,0
001001be,12,2,0,0,0,0,0,0,0
001802e4,2,0,0,0,0,0,0,0,0
0019e639,3,1,0,1,0,0,0,0,0


In [112]:
nuevoTrainingSet = nuevoTrainingSet.merge(dftmp, how='left', left_index=True, right_index=True)

nuevoTrainingSet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19414 entries, 0008ed71 to fff78145
Data columns (total 44 columns):
CantidadEventos              19414 non-null int64
CantidadCompras              19414 non-null float64
CantidadLeads                19414 non-null float64
VentasHaceUnMes              19414 non-null float64
LeadsHaceUnMes               19414 non-null float64
VentasHaceDosMeses           19414 non-null float64
LeadsHaceDosMeses            19414 non-null float64
CantidadCheckouts            19414 non-null float64
VentasMes1                   19414 non-null float64
VentasMes2                   19414 non-null float64
VentasMes3                   19414 non-null float64
VentasMes4                   19414 non-null float64
PromedioVentasPorMes         19414 non-null float64
DiasDesdeUltimoEvento        19414 non-null float64
DiasDesdePrimerEvento        19414 non-null float64
DiasDesdeUltimaConversion    19414 non-null float64
CantidadSearchEngineHits     19414 non-null float64
Entr

In [114]:
nuevoTrainingSet = nuevoTrainingSet.fillna(0)
nuevoTrainingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,sony,bcm,vam,vao,coam,coao,cam,cao,lam,lao
person,,,,,,,,,,,,,,,,,,,,,
0008ed71,6,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c79fe,17,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001802e4,19,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0019e639,471,2.0,0.0,1.0,0.0,1.0,0.0,15.0,0.0,1.0,...,0,3.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
001b0bf9,7,1.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [115]:
nuevoTestingSet = nuevoTestingSet.merge(dftmp, how='left', left_index=True, right_index=True)

nuevoTestingSet.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19415 entries, 00091926 to fffd1246
Data columns (total 44 columns):
CantidadEventos              19415 non-null int64
CantidadCompras              19415 non-null float64
CantidadLeads                19415 non-null float64
VentasHaceUnMes              19415 non-null float64
LeadsHaceUnMes               19415 non-null float64
VentasHaceDosMeses           19415 non-null float64
LeadsHaceDosMeses            19415 non-null float64
CantidadCheckouts            19415 non-null float64
VentasMes1                   19415 non-null float64
VentasMes2                   19415 non-null float64
VentasMes3                   19415 non-null float64
VentasMes4                   19415 non-null float64
PromedioVentasPorMes         19415 non-null float64
DiasDesdeUltimoEvento        19415 non-null float64
DiasDesdePrimerEvento        19415 non-null float64
DiasDesdeUltimaConversion    19415 non-null float64
CantidadSearchEngineHits     19415 non-null float64
Entr

In [116]:
nuevoTestingSet = nuevoTestingSet.fillna(0)
nuevoTestingSet.head()

,CantidadEventos,CantidadCompras,CantidadLeads,VentasHaceUnMes,LeadsHaceUnMes,VentasHaceDosMeses,LeadsHaceDosMeses,CantidadCheckouts,VentasMes1,VentasMes2,...,sony,bcm,vam,vao,coam,coao,cam,cao,lam,lao
person,,,,,,,,,,,,,,,,,,,,,
00091926,448,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00091a7a,10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000ba417,206,1.0,0.0,1.0,0.0,1.0,0.0,6.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000e4d9e,411,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000e619d,68,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
#nuevoTrainingSet.to_csv('trainingSet2.csv')
#Este comando reemplaza el training set existente con la feature agregada

In [118]:
#nuevoTestingSet.to_csv('testingSet2.csv')
#Este comando reemplaza el testing set existente con la feature agregada

In [83]:
df1 = pd.DataFrame({
    'person': [2, 3, 4, 5, 6, 7, 8, 10]})
df2 = pd.DataFrame({
    'person': [2, 4, 5, 7, 8],
    'apple': ['2.0', '0.0', '0.0', '5.0', '0.0']})
df1 = df1.merge(df2, on='person', how='left')
df1

,person,apple
0,2,2.0
1,3,NaN
2,4,0.0
3,5,0.0
4,6,NaN
5,7,5.0
6,8,0.0
7,10,NaN


In [ ]:
df1 = df1.merge(df2)

In [17]:
#nuevoTrainingSet.to_csv('trainingSet.csv')
#Este comando reemplaza el training set existente con la feature agregada

In [18]:
#nuevoTestingSet.to_csv('testingSet.csv')
#Este comando reemplaza el testing set existente con la feature agregada